In [1]:
import arcpy
import pandas as pd
import geopandas as gpd

C:\Users\ewiggans\AppData\Roaming\Python\Python39\site-packages\geopandas\_compat.py:123: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
C:\Users\ewiggans\AppData\Local\Temp\ipykernel_28312\1860243387.py:3: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://sh

In [2]:
# Create a geopandas data frame from the feature class
point_data = gpd.read_file(r"C:\Users\ewiggans\Desktop\GeoconnexNPDES\GeoconnexNPDES\data\NPDES_all.shp")

#view point data if you want
point_data.head()


,PERMANENT_,EVENTDATE,REACHCODE,REACHSMDAT,REACHRESOL,FEATURE_PE,FEATURECLA,SOURCE_ORI,SOURCE_DAT,SOURCE_FEA,...,START_DATE,END_DATE,WBD_HUC12,WBD_HUC12_,INNETWORK,NAVIGABLE,COASTAL,VERTICES,ESRI_KEY,geometry
0,{f09ef73c-f497-11ec-a765-eb4f65e0744a},2022-06-21,02080104000147,1999-08-13,3,8477146.00000000,0,DC,None,DCR10003Q,...,2022-06-21,None,020801040303,0.0,Y,Y,N,1,DCR10003Q,POINT (-8572064.700 4594781.225)
1,{f0888c1c-f497-11ec-a765-eb4f65e0744a},2022-06-21,02080104000899,1999-08-13,3,8477164.00000000,0,DC,None,DC0000221M13,...,2022-06-21,None,020801040303,0.0,Y,Y,N,1,DC0000221M13,POINT (-8575313.159 4594249.475)
2,{f0888c22-f497-11ec-a765-eb4f65e0744a},2022-06-21,02080104000669,1999-08-13,3,8476584.00000000,0,DC,None,DC0000299001,...,2022-06-21,None,020801040303,0.0,Y,Y,N,1,DC0000299001,POINT (-8571663.182 4592269.104)
3,{f0888c24-f497-11ec-a765-eb4f65e0744a},2022-06-21,02080104000669,1999-08-13,3,8476584.00000000,0,DC,None,DC0000299002,...,2022-06-21,None,020801040303,0.0,Y,Y,N,1,DC0000299002,POINT (-8571644.516 4592269.486)
4,{f0888c26-f497-11ec-a765-eb4f65e0744a},2022-06-21,02080104000669,1999-08-13,3,8474898.00000000,0,DC,None,DC0000299003,...,2022-06-21,None,020801040303,0.0,Y,Y,N,1,DC0000299003,POINT (-8571516.431 4592229.613)


In [3]:

# #########################
# # Export current DF to to Excel
# output_excel1 = r"C:\Users\ewiggans\Desktop\GeoconnexNPDES\GeoconnexNPDES\data\NPDES_point_data.xlsx"
# point_data.to_excel(output_excel1, index=False)

# print("Excel file exported successfully.")


In [4]:
# Specify the path to the Parquet file with the crosswalk of reachcode to comid
parquet_file = r'C:\Users\ewiggans\Desktop\GeoconnexNPDES\GeoconnexNPDES\scripts\nhdplusVAA.parquet'

# Load the crosswalk from the Parquet file
crosswalk = pd.read_parquet(parquet_file)

# Subset the crosswalk DataFrame to include only 'comid' and 'reachcode' columns
subset_crosswalk = crosswalk[['comid', 'reachcode', 'frommeas', 'tomeas']]

# Print the first few rows of the subset DataFrame if you want
#print(subset_crosswalk.head())

#need to delete trailing decimal zero on comid and then view dataframe head
crosswalk = subset_crosswalk.astype({'comid':'int32'})
crosswalk.head()


,comid,reachcode,frommeas,tomeas
0,8318793,18010102000885,0.0,100.0
1,8318787,18010102000886,0.0,100.0
2,8318775,18010102000887,0.0,100.0
3,8318785,18010102000888,0.0,100.0
4,8318789,18010102000889,0.0,100.0


In [5]:
#now join the parquet file crosswalk with the geopandas geodataframe 
# Perform the join operation
merged_data = point_data.merge(crosswalk, left_on='REACHCODE', right_on='reachcode')
merged_data = merged_data.rename(columns={'comid': 'comid_main'})

# View the merged data
print(merged_data.head())

print(merged_data.columns)


                               PERMANENT_   EVENTDATE       REACHCODE   
0  {f09ef73c-f497-11ec-a765-eb4f65e0744a}  2022-06-21  02080104000147  \
1  {f09ef73c-f497-11ec-a765-eb4f65e0744a}  2022-06-21  02080104000147   
2  {f09ef73c-f497-11ec-a765-eb4f65e0744a}  2022-06-21  02080104000147   
3  {f09ef73c-f497-11ec-a765-eb4f65e0744a}  2022-06-21  02080104000147   
4  {f0888c1c-f497-11ec-a765-eb4f65e0744a}  2022-06-21  02080104000899   

   REACHSMDAT  REACHRESOL        FEATURE_PE  FEATURECLA SOURCE_ORI SOURCE_DAT   
0  1999-08-13           3  8477146.00000000           0         DC       None  \
1  1999-08-13           3  8477146.00000000           0         DC       None   
2  1999-08-13           3  8477146.00000000           0         DC       None   
3  1999-08-13           3  8477146.00000000           0         DC       None   
4  1999-08-13           3  8477164.00000000           0         DC       None   

     SOURCE_FEA  ... INNETWORK  NAVIGABLE  COASTAL  VERTICES      ESRI_KEY

In [6]:
# #########################
# # Export df after first join
# output_excel2 = r"C:\Users\ewiggans\Desktop\GeoconnexNPDES\GeoconnexNPDES\data\NPDES_after_first_join.xlsx"
# merged_data.to_excel(output_excel2, index=False)

# print("Excel file exported successfully.")

In [7]:
# Specify the path to the CSV file for the second join
csv_file = r'C:\Users\ewiggans\Desktop\GeoconnexNPDES\GeoconnexNPDES\scripts\nhdpv2_lookup.csv'

# Load the CSV file
csv_data = pd.read_csv(csv_file)

# Rename the 'uri' field in the CSV to 'mainstem_uri'
csv_data = csv_data.rename(columns={'uri': 'mainstem_uri'})

# Perform the second join operation based on 'comid'
selected_data = merged_data.merge(csv_data, left_on='comid_main', right_on='comid', how='left')

# Print the head of the selected_data DataFrame
print(selected_data.head())

# Print the column names of the selected_data DataFrame
print(selected_data.columns)


                               PERMANENT_   EVENTDATE       REACHCODE   
0  {f09ef73c-f497-11ec-a765-eb4f65e0744a}  2022-06-21  02080104000147  \
1  {f09ef73c-f497-11ec-a765-eb4f65e0744a}  2022-06-21  02080104000147   
2  {f09ef73c-f497-11ec-a765-eb4f65e0744a}  2022-06-21  02080104000147   
3  {f09ef73c-f497-11ec-a765-eb4f65e0744a}  2022-06-21  02080104000147   
4  {f0888c1c-f497-11ec-a765-eb4f65e0744a}  2022-06-21  02080104000899   

   REACHSMDAT  REACHRESOL        FEATURE_PE  FEATURECLA SOURCE_ORI SOURCE_DAT   
0  1999-08-13           3  8477146.00000000           0         DC       None  \
1  1999-08-13           3  8477146.00000000           0         DC       None   
2  1999-08-13           3  8477146.00000000           0         DC       None   
3  1999-08-13           3  8477146.00000000           0         DC       None   
4  1999-08-13           3  8477164.00000000           0         DC       None   

     SOURCE_FEA  ... COASTAL  VERTICES      ESRI_KEY   
0     DCR10003Q  .

In [8]:

# #########################
# # Export DF after second join
# output_excel3 = r"C:\Users\ewiggans\Desktop\GeoconnexNPDES\GeoconnexNPDES\data\NPDES_after_second_join.xlsx"
# selected_data.to_excel(output_excel3, index=False)

# print("Excel file exported successfully.")


In [9]:
# Filter the merged data based on MEASURE being less than to measure
filtered_data = selected_data[selected_data['MEASURE'] <= selected_data['tomeas']]

# View the filtered data
print(filtered_data.head())

#print column names
column_names = ", ".join(filtered_data.columns)
print(column_names)


                               PERMANENT_   EVENTDATE       REACHCODE   
1  {f09ef73c-f497-11ec-a765-eb4f65e0744a}  2022-06-21  02080104000147  \
5  {f0888c1c-f497-11ec-a765-eb4f65e0744a}  2022-06-21  02080104000899   
6  {f0888c22-f497-11ec-a765-eb4f65e0744a}  2022-06-21  02080104000669   
8  {f0888c22-f497-11ec-a765-eb4f65e0744a}  2022-06-21  02080104000669   
9  {f0888c22-f497-11ec-a765-eb4f65e0744a}  2022-06-21  02080104000669   

   REACHSMDAT  REACHRESOL        FEATURE_PE  FEATURECLA SOURCE_ORI SOURCE_DAT   
1  1999-08-13           3  8477146.00000000           0         DC       None  \
5  1999-08-13           3  8477164.00000000           0         DC       None   
6  1999-08-13           3  8476584.00000000           0         DC       None   
8  1999-08-13           3  8476584.00000000           0         DC       None   
9  1999-08-13           3  8476584.00000000           0         DC       None   

     SOURCE_FEA  ... COASTAL  VERTICES      ESRI_KEY   
1     DCR10003Q  .

In [10]:

# #########################################
# #Export after Filter
# output_excel4 = r"C:\Users\ewiggans\Desktop\GeoconnexNPDES\GeoconnexNPDES\data\NPDES_filtered.xlsx"
# filtered_data.to_excel(output_excel4, index=False)

# print("Excel file exported successfully.")

In [11]:
#Get unique Rows
remove_dup_data = filtered_data.drop_duplicates()

#Export after unique rows 
# output_excel5 = r"C:\Users\ewiggans\Desktop\GeoconnexNPDES\GeoconnexNPDES\data\NPDES_remove_dup.xlsx"
# filtered_data.to_excel(output_excel5, index=False)

# print("Excel file exported successfully.")

In [12]:
# Select specific columns and add new fields as a GeoDataFrame
selected_data = gpd.GeoDataFrame(filtered_data[['REACHCODE', 'SOURCE_FEA', 'FEATUREDET', 'MEASURE', 'comid_main','mainstem_uri', 'geometry']])
# Rename the 'comid_main' field 
selected_data = selected_data.rename(columns={'comid_main': 'comid'})
selected_data['uri'] = 'https://geoconnex.us/iow/npdes/' + selected_data['SOURCE_FEA']
selected_data['id'] = selected_data['SOURCE_FEA']
selected_data['name'] = selected_data['SOURCE_FEA']
print("Renaming and additional columns complete")

Renaming and additional columns complete


In [13]:
# Export selected_data to a shapefile
output_shapefile = r"C:\Users\ewiggans\Desktop\GeoconnexNPDES\GeoconnexNPDES\data\NPDES_all_output.shp"
selected_data.to_file(output_shapefile)

print("Shapefile exported successfully.")

C:\Users\ewiggans\AppData\Local\Temp\ipykernel_28312\1782497313.py:3: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  selected_data.to_file(output_shapefile)


Shapefile exported successfully.


In [15]:
# Export selected_data to a geopackage
output_geopackage = r"C:\Users\ewiggans\Desktop\GeoconnexNPDES\GeoconnexNPDES\data\NPDES_all_output.gpkg"
selected_data.to_file(output_geopackage, layer='NPDES_subset', driver='GPKG')

print("Geopackage exported successfully.")

Geopackage exported successfully.
